In [1]:
!pip install confluent_kafka sseclient geopandas

  Using cached confluent_kafka-2.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached sseclient-0.0.27-py3-none-any.whl
  Using cached geopandas-0.14.4-py3-none-any.whl.metadata (1.5 kB)
  Using cached fiona-1.9.6-cp311-cp311-manylinux2014_x86_64.whl.metadata (50 kB)
  Using cached pyproj-3.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached shapely-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
Using cached confluent_kafka-2.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.1 MB)
Using cached geopandas-0.14.4-py3-none-any.whl (1.1 MB)
Using cached fiona-1.9.6-cp311-cp311-manylinux2014_x86_64.whl (15.7 MB)
Using cached pyproj-3.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cached shape

In [2]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import os
import sys, random

brokers = os.getenv('KAFKA_ADVERTISED_LISTENERS')
protocol = os.getenv('KAFKA_LISTENER_SECURITY_PROTOCOL_MAP')

In [3]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer': StringSerializer('utf_8'),
    'security.protocol': 'PLAINTEXT'
}

In [4]:
p = SerializingProducer(pconf)

In [5]:
stocks = "sample.csv"

In [6]:
from datetime import datetime
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, Point

# Load GeoJSON file into GeoDataFrame
gdf = gpd.read_file('./nyc-boroughs.geojson')
# Ensure geometries are valid
gdf['geometry'] = gdf['geometry'].apply(lambda x: x if x.is_valid else x.buffer(0))

# Create a list of tuples (boroughCode, geometry)
boroughs = gdf.apply(lambda row: (row['boroughCode'], shape(row['geometry'])), axis=1).tolist()

# Define the function to get borough code based on coordinates
def get_borough(longitude, latitude):
    point = Point(longitude, latitude)
    for code, geom in boroughs:
        if geom.contains(point):
            return code
    return None


def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    stock = {
        #  Some Unique ID for the car (license)
             "hack_licence": row[1],
        #  Pick-up location
             "pickup_location": get_borough(row[10], row[11]),
        #  Drop-off location
             "drop_off_location": get_borough(row[12], row[13]),
        #  Pick-up time
              "pick_up_time": row[5],
        #  Drop-off time
              "drop_off_time": row[6],
              "timestamp":str_date_time
            }
    return stock

In [ ]:
import csv, json
import time
n = 0
with open(stocks) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # Skip the header row
    next(csv_reader, None)
    try:
        for row in csv_reader:
            #if n == 100:
             #  break
            stock = construct_stock(row)
            print(stock)
            p.produce('stock', value=json.dumps(stock))
            p.poll(0)
            p.flush()
            time.sleep(0.5)
            #n = n + 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

    
        
    
    
    

{'hack_licence': 'BA96DE419E711691B9445D6A6307C170', 'pickup_location': 1, 'drop_off_location': 1, 'pick_up_time': '2013-01-01 15:11:48', 'drop_off_time': '2013-01-01 15:18:10', 'timestamp': '2024-05-23T17:10:26Z'}
{'hack_licence': '9FD8F69F0804BDB5549F40E9DA1BE472', 'pickup_location': 1, 'drop_off_location': 1, 'pick_up_time': '2013-01-06 00:18:35', 'drop_off_time': '2013-01-06 00:22:54', 'timestamp': '2024-05-23T17:10:28Z'}
{'hack_licence': '9FD8F69F0804BDB5549F40E9DA1BE472', 'pickup_location': 1, 'drop_off_location': 1, 'pick_up_time': '2013-01-05 18:49:41', 'drop_off_time': '2013-01-05 18:54:23', 'timestamp': '2024-05-23T17:10:28Z'}
{'hack_licence': '51EE87E3205C985EF8431D850C786310', 'pickup_location': 1, 'drop_off_location': 1, 'pick_up_time': '2013-01-07 23:54:15', 'drop_off_time': '2013-01-07 23:58:20', 'timestamp': '2024-05-23T17:10:29Z'}
{'hack_licence': '51EE87E3205C985EF8431D850C786310', 'pickup_location': 1, 'drop_off_location': 1, 'pick_up_time': '2013-01-07 23:25:03', 'd

### Use only if you need to purge all the messages in the queue

In [ ]:
brokers = "broker:9092"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=['stock'])